In [ ]:
# Cell-Cell direct contact detection of cellular junctions
# Find boundaries and retrieve signal at border/background region
# between every pair of cell in contact 
# authors: Pacome Prompsy
# contact: pacome.prompsy@chuv.ch
# Guenova Lab
# CHUV (Centre Hospitalier Universitaire Vaudois), Lausanne, Suisse

In [ ]:
import numpy as np
from skimage import measure
from tifffile import imread, imwrite
import os
import pandas as pd
import numpy as np
from tifffile import imread, imwrite
from skimage.morphology import label
from skimage.segmentation import expand_labels
from concurrent.futures import ThreadPoolExecutor
from skimage.morphology import dilation
from scipy.stats import fisher_exact
from scipy.stats import ttest_ind
import pickle

In [ ]:
from ark.segmentation import fiber_segmentation, marker_quantification, regionprops_extraction, signal_extraction 

In [ ]:
output_dir = "../output/cell_neighborhood/interactions/"
cell_marker_file = "../annotation/cell_markers.csv"
tiff_dir = "../output/input/"
segmentation_dir  = "../output/segmentation/"
tiff_dir = "../output/input/"
marker_metadata = pd.read_csv("../annotation/marker_metadata.csv")
sample_metadata = pd.read_csv("../annotation/Sample_metadata.csv", sep=";")

In [ ]:
samples = sample_metadata.ROI[sample_metadata.KeepForAnalysis == True & 
                              np.in1d(sample_metadata.Disease, ["LN", "TON", "THY"], invert=True)].values

In [ ]:
markers = np.array(marker_metadata.Marker[marker_metadata.PassOverallQuality == True])
markers = markers[markers != "Ki67"]
markers = markers[markers != "CXCR4"]

In [ ]:
def enrich_interactions(sample, segmentation_dir,output_dir,  tiff_dir, markers):
    print("Running sample " + sample + " ...")

    # Read whole cell and boundaries
    seg_image = imread(os.path.join(segmentation_dir, sample + "_whole_cell.tiff"))
    seg_boundary = imread(os.path.join(segmentation_dir, sample + "_whole_cell_segmentation_borders.tiff"))
    seg_boundary = label(seg_boundary)
    seg_boundary = expand_labels(seg_boundary, 1)
    seg_image_boundary = seg_image.copy()
    seg_image_boundary[seg_boundary==0] = 0

    seg_image_dilated = seg_image_boundary.copy()
    dilation(seg_image_dilated, out = seg_image_dilated)
    seg_image_expanded = seg_image_boundary.copy()
    seg_image_expanded = expand_labels(label(seg_image_expanded), 1)
    dict_inter = {}

    cells = np.unique(seg_image)
    cells = cells[cells>0]

    print("Reading images...")
    image_dict = {}
    for j,item in enumerate(markers):
                marker = markers[j]

                image_dict[marker] = imread(os.path.join(tiff_dir, sample, marker + ".tiff"))
    
    
    print("Finding interactions for " + str(len(cells)) + " cells...")

    for cell in cells:
        if cell % 100 == 0:
            print(sample + " - " + str(cell)) 
            
        seg_image_expanded_cell = seg_image_expanded.copy()
        seg_image_expanded_cell[seg_image_boundary != cell] = 0

        image_cell = seg_image.copy()
        image_cell[image_cell != cell] = 0

        intersection = np.unique(seg_image_dilated[seg_image_expanded_cell>0])
        intersection = intersection[(intersection != cell) & (intersection != 0)]
        
        #print("Found " + str(len(intersection)) + " intersections...")
        if len(intersection) > 0:
            area_inter_dict = {}
            for i in intersection:

                # select other cell whole cell
                image_other_cell = seg_image.copy()
                image_other_cell[image_other_cell != i] = 0

                # Create expanded border
                border = seg_image_dilated.copy()
                border[(seg_image_expanded_cell != 0) & (seg_image_dilated == i)] = -1
                border[border != -1] = 0
                border[border == -1] = 1

                # Create background from the border, just more expanded
                background = border.copy()
                
                border = expand_labels(border, 8)
                
                background = expand_labels(border, 15)

                # Filter border for the cell or the other cell only
                both_cells = (image_cell + image_other_cell)
                border[(both_cells == 0)] = 0

                # Separate background into belonging to cell or other cell
                background_cell = background.copy()
                background_cell[image_cell == 0] = 0
                background_cell[border == 1] = 0
                
                background_other_cell = background.copy()
                background_other_cell[image_other_cell == 0] = 0
                background_other_cell[border == 1] = 0

                results_count = np.zeros((len(markers),5))
                for j,item in enumerate(markers):
                    marker = markers[j]
                    
                    marker_image = image_dict[marker]

                    sample1 = marker_image[border > 0]
                    sample2 = marker_image[background_cell > 0]
                    sample3 = marker_image[background_other_cell > 0]
                    t_stat, p_value = ttest_ind(sample1, sample2, alternative= "greater")
                    results_count[j,0] = p_value
                    t_stat, p_value = ttest_ind(sample1, sample3, alternative= "greater")
                    results_count[j,1] = p_value
                    mean_border = np.mean(sample1)
                    mean_background = ((np.mean(sample2) + np.mean(sample3)) /2)
                    FC = mean_border / mean_background

                    results_count[j,2] = mean_border
                    results_count[j,3] = mean_background
                    results_count[j,4] = FC

                area_inter_dict[i] = results_count

            dict_inter[str(cell)] = area_inter_dict

    
    with open("../output/cell_neighborhood/enriched_interactions/dict_" + sample + ".pkl", 'wb') as fp:
        pickle.dump(dict_inter, fp)
        print(sample + ' finished and saved successfully !')
    
    return(True)

In [ ]:
# Create a ThreadPoolExecutor with max_workers set to the number of threads you want to use
with ThreadPoolExecutor(max_workers=12) as executor:
    # Submit the tasks with multiple arguments using *args
    for sample in samples:
        executor.submit(enrich_interactions, sample, segmentation_dir, output_dir,  tiff_dir, markers)


In [ ]:


# Prepare interaction dataframe
all_cells = np.unique(seg_image)
all_cells = all_cells[all_cells !=0]
n_cells = len(all_cells)
touching_matrix = np.zeros((n_cells, n_cells), dtype=int)
touching_matrix = pd.DataFrame(touching_matrix)
touching_matrix.columns = all_cells.astype(str)
touching_matrix.index = all_cells.astype(str)

print("Filling DF " + sample + "...")

# Fill interaction data frame
for j, (k, v) in enumerate(dict_inter.items()):
    for j2, (k2, v2) in enumerate(v.items()):
        touching_matrix.loc[str(k), str(k2)] = v2

# Render symmetric
for i in touching_matrix.columns:
    for j in touching_matrix.index:
        if touching_matrix.loc[i,j] != touching_matrix.loc[j,i]:
            if touching_matrix.loc[i,j] == 0:
                touching_matrix.loc[i,j] = touching_matrix.loc[j,i]
            else:
                touching_matrix.loc[j,i] = touching_matrix.loc[i,j]

print("Writing DF" + sample +"...")
touching_matrix.to_csv(os.path.join(output_dir, sample + ".csv.gz"), compression='gzip')

return(True)